In [27]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time

In [98]:
url=['UNH','JNJ','LLY','ABBV','PFE','MRK','TMO','ABT','DHR','BMY', 'AMGN', 'MDT', 'BSX', 'REGN', 'HCA', 'GILD', 'ZTS', 'MCK', 'CSV', 'BDX','EW', 'DXCM', 'A', 'IDXX', 'COR', 'IQV', 'CNC', 'GEHC','BIIB', 'MTD', 'RMD', 'WST','ZBH']
tipo_estado=['financials','balance-sheet','cash-flow']
base='https://es-us.finanzas.yahoo.com/quote/'


for i in tipo_estado:
    for j in url:
        labels=["Descripcion"]
        values=["Valor"]
        link=base+j+'/'+i
        req=Request(link, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        bs = BeautifulSoup(webpage, 'html.parser')
        soup=bs.find_all('div',class_='D(tbr) fi-row Bgc($hoverBgColor):h')
        for s in soup:
            label=s.find('div', class_='Va(m)')
            value=s.find('div', class_='Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)')
            if value.text =='':
                value=0
            else:
                value=value.text.replace(',','')
                value=value.replace('-','0')
                value=float(value)
            labels.append(label.text)
            values.append(value)
        df = pd.DataFrame({labels[0]: labels[1:], values[0]: values[1:]})
        df.to_csv(j+'_'+i+'.csv', index=False)
    


In [101]:
import csv
import os

empresas = ['UNH','JNJ','LLY','ABBV','PFE','MRK','TMO','ABT','DHR','BMY', 'AMGN', 'MDT', 'BSX', 'REGN', 'HCA', 'GILD', 'ZTS', 'MCK', 'CSV', 'BDX','EW', 'DXCM', 'A', 'IDXX', 'COR', 'IQV', 'CNC', 'GEHC','BIIB', 'MTD', 'RMD', 'WST','ZBH']

# Función para leer datos de un archivo CSV
def read_csv(filename):
    data = {}
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Saltar la primera fila (encabezados)
        for row in reader:
            key = row[0]
            value = float(row[1])
            data[key] = value
    return data

# Función para manejar la división entre cero
def safe_divide(numerator, denominator):
    if denominator == 0:
        return "N/A"
    else:
        return numerator / denominator

# Calcular razones financieras
def calculate_ratios(balance_sheet, cash_flow, financials):
    ratios = {}

    # Razones del balance general
    ratios['EfecAct'] = safe_divide(balance_sheet['Caja y equivalentes de caja'], balance_sheet['Activos totales'])
    ratios['CliAct'] = safe_divide(balance_sheet['Cuentas pendientes netas'], balance_sheet['Activos totales'])
    ratios['InvAct'] = 0  # No hay inventarios en el balance
    ratios['AcAcT'] = safe_divide(balance_sheet['Activos corrientes totales'], balance_sheet['Activos totales'])
    ratios['InmAct'] = safe_divide(balance_sheet['Propiedad neta, planta y equipo'], balance_sheet['Activos totales'])
    ratios['OaAct'] = safe_divide(balance_sheet['Otros activos a largo plazo'], balance_sheet['Activos totales'])
    ratios['PacAct'] = safe_divide(balance_sheet['Pasivos circulantes totales'], balance_sheet['Activos totales'])
    ratios['PrsAct'] = safe_divide(balance_sheet['Deuda a largo plazo'], balance_sheet['Activos totales'])
    ratios['PalAct'] = safe_divide(balance_sheet['Total de obligaciones no corrientes'], balance_sheet['Activos totales'])
    ratios['TopAct'] = safe_divide(balance_sheet['Pasivos totales'], balance_sheet['Activos totales'])
    ratios['tocact'] = safe_divide(balance_sheet['Total de patrimonio neto de los accionistas'], balance_sheet['Activos totales'])

    # Razones del estado de resultados
    ratios['cosvta'] = safe_divide(financials['Costo de ganancias'], financials['Ingresos totales'])
    ratios['uopvta'] = safe_divide(financials['Ingresos o pérdidas operativos'], financials['Ingresos totales'])
    ratios['gfinvta'] = safe_divide(financials['Gastos de interés'], financials['Ingresos totales'])
    ratios['uaivta'] = safe_divide(financials['Ingreso antes de impuestos'], financials['Ingresos totales'])
    ratios['impvta'] = safe_divide(financials['Impuesto a las ganancias'], financials['Ingresos totales'])
    ratios['utnvta'] = safe_divide(financials['Ingreso neto disponible para accionistas comunes'], financials['Ingresos totales'])

    # Razones de liquidez y actividad
    ratios['Acpc'] = safe_divide(balance_sheet['Activos corrientes totales'], balance_sheet['Pasivos circulantes totales'])
    ratios['Acinvpc'] = safe_divide(balance_sheet['Activos corrientes totales'] - ratios['InvAct'], balance_sheet['Pasivos circulantes totales'])
    ratios['rotinv'] = safe_divide(financials['Ingresos totales'], ratios['InvAct'])
    ratios['rotcxc'] = safe_divide(financials['Ingresos totales'], balance_sheet['Cuentas pendientes netas'])
    ratios['rotact'] = safe_divide(financials['Ingresos totales'], balance_sheet['Activos totales'])
    ratios['intact'] = safe_divide(balance_sheet['Activos totales'], financials['Ingresos totales'])

    # Razones de apalancamiento
    ratios['pascc'] = safe_divide(balance_sheet['Pasivos totales'], balance_sheet['Total de patrimonio neto de los accionistas'])
    ratios['multcap'] = safe_divide(1, ratios['tocact'])
    ratios['paslpcc'] = safe_divide(balance_sheet['Total de obligaciones no corrientes'], balance_sheet['Total de patrimonio neto de los accionistas'])
    ratios['cobinter'] = safe_divide(financials['Ingresos o pérdidas operativos'], financials['Gastos de interés'])

    # Razones de rentabilidad
    ratios['roe'] = safe_divide(financials['Ingreso neto disponible para accionistas comunes'], balance_sheet['Total de patrimonio neto de los accionistas'])
    ratios['roa'] = safe_divide(financials['Ingreso neto disponible para accionistas comunes'], balance_sheet['Activos totales'])
    ratios['ros'] = safe_divide(financials['Ingreso neto disponible para accionistas comunes'], financials['Ingresos totales'])
    ratios['upa'] = safe_divide(financials['Ingreso neto disponible para accionistas comunes'], financials['Promedio de acciones diluidas'])

    # Razones no calculadas por falta de datos
    ratios['diainv'] = 0
    ratios['diacxc'] = 0

    return ratios

# Crear una carpeta para almacenar los archivos CSV de razones financieras
os.makedirs('financial_ratios', exist_ok=True)

# Iterar por cada empresa
for empresa in empresas:
    # Leer datos de los archivos CSV
    balance_sheet = read_csv(f'{empresa}_balance-sheet.csv')
    cash_flow = read_csv(f'{empresa}_cash-flow.csv')
    financials = read_csv(f'{empresa}_financials.csv')

    # Calcular razones financieras
    financial_ratios = calculate_ratios(balance_sheet, cash_flow, financials)

    # Crear un archivo CSV para las razones financieras de la empresa
    with open(f'financial_ratios/{empresa}_financial_ratios.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Razón', 'Valor'])
        for ratio, value in financial_ratios.items():
            writer.writerow([ratio, value])